# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder

data_path = 'data/Part02/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?


In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.780004837244799
time : 0.046865224838256836 sec


In [4]:
# 均值編碼 + 邏輯斯迴歸
data = pd.concat([df[:train_num], train_Y], axis=1)
for c in df.columns:
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['Survived'] , axis=1)
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 1.0
time : 0.024933338165283203 sec


In [5]:
# Label Encoder & One Hot Encoder & Mean Encoder 
fill_in = ['LabelEncoder_LinearRegression','LabelEncoder_LogisticRegression','LabelEncoder_GradiantBoostingRegres',
           'OneHotEncoder_LinearRegression','OneHotEncoder_LogisticRegression','OneHotEncoder_GradiantBoostingRegres',
           'MeanEncoder_LinearRegression', 'MeanEncoder_LogisticRegression', 'MeanEncoder_GradiantBoostingRegres']
scaling = ['Shape','Score','Time']
result_set = {}

df_temp = pd.DataFrame()
df_temp2 = pd.get_dummies(df)
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
train_X2 = df_temp2[:train_num]
data = pd.concat([df[:train_num], train_Y], axis=1)
for c in df.columns:
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['Survived'] , axis=1)


for fill in fill_in:
    result = []
    if 'LabelEncoder' in fill:
        if fill == 'LabelEncoder_LinearRegression':
            estimator = LinearRegression()
            start = time.time()         
        elif fill == 'LabelEncoder_LogisticRegression':
            estimator = LogisticRegression()
            start = time.time()        
        elif fill == 'LabelEncoder_GradiantBoostingRegres':
            estimator = GradientBoostingRegressor()
            start = time.time()
        result.append(train_X.shape)
        result.append(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
        result.append(time.time() - start)               
    if 'OneHotEncoder' in fill:
        if fill == 'OneHotEncoder_LinearRegression':
            estimator = LinearRegression()
            start = time.time()         
        elif fill == 'OneHotEncoder_LogisticRegression':
            estimator = LogisticRegression()
            start = time.time()        
        elif fill == 'OneHotEncoder_GradiantBoostingRegres':
            estimator = GradientBoostingRegressor()
            start = time.time()
        result.append(train_X2.shape)
        result.append(cross_val_score(estimator, train_X2, train_Y, cv=5).mean())
        result.append(time.time() - start)
    if 'MeanEncoder' in fill:
        if fill == 'MeanEncoder_LinearRegression':
            estimator = LinearRegression()
            start = time.time()         
        elif fill == 'MeanEncoder_LogisticRegression':
            estimator = LogisticRegression()
            start = time.time()        
        elif fill == 'MeanEncoder_GradiantBoostingRegres':
            estimator = GradientBoostingRegressor()
            start = time.time()           
        result.append(train_X.shape)
        result.append(cross_val_score(estimator, data, train_Y, cv=5).mean())
        result.append(time.time() - start)
    result_set[fill] = result

result_df = pd.DataFrame(result_set,index=scaling)
result_df

,LabelEncoder_GradiantBoostingRegres,LabelEncoder_LinearRegression,LabelEncoder_LogisticRegression,MeanEncoder_GradiantBoostingRegres,MeanEncoder_LinearRegression,MeanEncoder_LogisticRegression,OneHotEncoder_GradiantBoostingRegres,OneHotEncoder_LinearRegression,OneHotEncoder_LogisticRegression
Shape,"(891, 5)","(891, 5)","(891, 5)","(891, 5)","(891, 5)","(891, 5)","(891, 2429)","(891, 2429)","(891, 2429)"
Score,0.377196,0.334943,0.780005,1,1,1,0.375828,0.416473,0.801335
Time,0.362033,0.0169549,0.0339096,0.0937507,0.00897646,0.0129645,7.62262,2.14825,0.202459


Ans: As result, Mean encoder method is better than Label encoder method.